In [1]:
# DONE-read in each dataset
# DONE-change column headers on each
# DONE-copy df and append all sheets
# DONE-clean up data
# DONE-change all negatives to absolute values
# DONE-dtype columns to floats
# DONE-add lookup for siteid for consistency
# DONE-get lat/long for each location from kmz file
# DONE-add a lookup for lat/long
# DONE-melt crosstab df to a flat file
# DONE-create stats table for each site and each analyte (incl mean, 20%, 80%, n, but also min, max, median, skew, kurtosis, etc)
# add _mg/l or other to new columns
# convert DisAl,DisAs, DisB, DisCd, DisCr, DisCu, DisFe, DisPb, DisNi, DisSe, DisAg, DisZn to all mg/l not ug/l

# DONE-add a map of all sites (color=mean, size=stdev or n)
# DONE-make a bar plot of all sites by distance downstream incl below dam
# DONE-make violin plots for each site (for TDS, TotFe, DisFe)
# DONE-add time trend comparisons

### Package Dependencies to run analysis

In [2]:
import numpy as np
import pandas as pd

import plotly.graph_objs as go
import chart_studio.plotly as py
import plotly.figure_factory as ff
from plotly.subplots import make_subplots
# py.tools.set_credentials_file(username='cshope', api_key='mZDVXt9g94uZZQShyop2')
# py.sign_in('cshope', 'mZDVXt9g94uZZQShyop2')

### Read in the Global Inputs or the raw data and clean up

In [3]:
# Read the ElectricLakePacifiCorpData2009-2013.xlsx data
column_list = []
PC_cross_cols = pd.read_excel("ElectricLakePacifiCorpData2009-2013.xlsx", sheet_name="PacifiCorp_Data", 
                              skiprows=0, header=1).columns
for i in PC_cross_cols:
    column_list.append(i) # list of column headers
converter = {col: float for col in column_list[2:]} # convert most cols to float 
blanks = ('',' ','NA','#N/A','.','*','--','?','a','n','na','n/a','n.a.','n.d.',
          'nd','nm','nr','ns','x','Err','MD','N','NC','NA','ND','NDD','NE','NM',
          'NS','N/A','#VALUE!','not in','not  in','nm')
df_PC_cross = pd.read_excel("ElectricLakePacifiCorpData2009-2013.xlsx", sheet_name="PacifiCorp_Data", skiprows=0,
                   header=1, converters=converter, na_values=blanks)

old_colnames = ('Client Ident','Started',
 '0751\nMG/L\nOxygen, Dissolved\nOxygen, Dissolved\nMilligrams per Liter',
 '0751\nMG/L\nOil and Grease (HEM)\nOil and Grease, (HEM)\nMilligrams per Liter',
 '0752\nMG/L\nNitrogen, Ammonia\nNitrogen, Ammonia\nMilligrams per Liter',
 '9999\nS.U.\npH\npH\nStandard Units',
 '0844\nMG/L\nSolids, Total Dissolved\nTotal Dissolved Solids\nMilligrams per Liter',
 '0844\nMG/L\nSolids, Total Suspended\nTotal Suspended Solids\nMilligrams per Liter',
 '0751\nMG/L\nInorganic Anions\nNitrate\nMilligrams per Liter',
 '0751\nMG/L\nInorganic Anions\nNitrite\nMilligrams per Liter',
 'Nitrate + Nitrate as Nitrogen',
 '5531\nUG/L\nMercury, Hg\nMercury, Hg - Total\nMicrograms per Liter',
 '5531\nUG/L\nMercury, Hg\nMercury, Hg - Dissolved\nMicrograms per Liter',
 '5651\nMG/L\nPhosphorus, P\nPhosphorus, Dissolved\nMilligrams per Liter',
 '5651\nMG/L\nPhosphorus, P\nPhosphorus, Total\nMilligrams per Liter',
 '5021\nMG/L\nAqueous Metals\nAluminum, Al - Total\nMilligrams per Liter',
 '5021\nMG/L\nAqueous Metals\nAluminum, Al - Dissolved\nMilligrams per Liter',
 '5061\nMG/L\nAqueous Metals\nArsenic, As - Total\nMilligrams per Liter',
 '5061\nMG/L\nAqueous Metals\nArsenic, As - Dissolved\nMilligrams per Liter',
 '0512\nMG/L\nAqueous Metals\nBoron, B - Total\nMilligrams per Liter',
 '0012\nMG/L\nAqueous Metals\nBoron, B - Dissolved\nMilligrams per Liter',
 '5141\nMG/L\nAqueous Metals\nCadmium, Cd - Total\nMilligrams per Liter',
 '5141\nMG/L\nAqueous Metals\nCadmium, Cd - Dissolved\nMilligrams per Liter',
 '5151\nMG/L\nAqueous Metals\nCalcium, Ca - Total\nMilligrams per Liter',
 '5151\nMG/L\nAqueous Metals\nCalcium, Ca - Dissolved\nMilligrams per Liter',
 '5211\nMG/L\nAqueous Metals\nChromium, Cr - Total\nMilligrams per Liter',
 '5211\nMG/L\nAqueous Metals\nChromium, Cr - Dissolved\nMilligrams per Liter',
 '5231\nMG/L\nAqueous Metals\nCopper, Cu - Total\nMilligrams per Liter',
 '5231\nMG/L\nAqueous Metals\nCopper, Cu - Dissolved\nMilligrams per Liter',
 '5431\nMG/L\nAqueous Metals\nIron, Fe - Total\nMilligrams per Liter',
 '5431\nMG/L\nAqueous Metals\nIron, Fe - Dissolved\nMilligrams per Liter',
 '5471\nMG/L\nAqueous Metals\nLead, Pb - Total\nMilligrams per Liter',
 '5471\nMG/L\nAqueous Metals\nLead, Pb - Dissolved\nMilligrams per Liter',
 '5501\nMG/L\nAqueous Metals\nMagnesium, Mg - Total\nMilligrams per Liter',
 '5501\nMG/L\nAqueous Metals\nMagnesium, Mg - Dissolved\nMilligrams per Liter',
 '5581\nMG/L\nAqueous Metals\nNickel, Ni - Total\nMilligrams per Liter',
 '5581\nMG/L\nAqueous Metals\nNickel, Ni - Dissolved\nMilligrams per Liter',
 '5691\nMG/L\nAqueous Metals\nPotassium, K - Total\nMilligrams per Liter',
 '5691\nMG/L\nAqueous Metals\nPotassium, K - Dissolved\nMilligrams per Liter',
 '5811\nMG/L\nAqueous Metals\nSelenium, Se - Total\nMilligrams per Liter',
 '5811\nMG/L\nAqueous Metals\nSelenium, Se - Dissolved\nMilligrams per Liter',
 '5831\nMG/L\nAqueous Metals\nSilver, Ag - Total\nMilligrams per Liter',
 '5831\nMG/L\nAqueous Metals\nSilver, Ag - Dissolved\nMilligrams per Liter',
 '6021\nMG/L\nAqueous Metals\nZinc, Zn - Total\nMilligrams per Liter',
 '6021\nMG/L\nAqueous Metals\nZinc, Zn - Dissolved\nMilligrams per Liter')

new_colnames =('ClientID', 'Started', 'DO', 'O&G Hem', 'NH3-N', 'ph', 
 'TDS', 'TSS', 'NO3', 'NO2', 'NO2_3-N', 'TotHg', 'DisHg', 'DisP', 'TotP', 'TotAl', 'DisAl', 'TotAs',
 'DisAs', 'TotB', 'DisB', 'TotCd', 'DisCd', 'TotCa', 'DisCa', 'TotCr', 'DisCr', 'TotCu', 'DisCu',
 'TotFe', 'DisFe', 'TotPb', 'DisPb', 'TotMg', 'DisMg', 'TotNi', 'DisNi', 'TotK', 'DisK', 'TotSe',
 'DisSe', 'TotAg', 'DisAg', 'TotZn', 'DisZn') # 'Temp', 'Source', 'LabId', 'SampleType', 'Client', 

df_PC_cross.rename(columns={i:j for i,j in zip(old_colnames,new_colnames)}, inplace=True) # change the column headers
cols = df_PC_cross.columns.drop('ClientID') # since string, keep col as object
df_PC_cross[cols] = df_PC_cross[cols].apply(pd.to_numeric, errors='coerce') # convert cols to floats, not objects
df_PC_cross['Started'] = pd.to_datetime(df_PC_cross['Started']) # Convert Date column to datetime
    
df_PC_cross

,ClientID,Started,DO,O&G Hem,NH3-N,ph,TDS,TSS,NO3,NO2,...,TotNi,DisNi,TotK,DisK,TotSe,DisSe,TotAg,DisAg,TotZn,DisZn
0,H-1 (HTG Creek @ River Intake),2009-12-06 00:00:00,6.60,-5.0000,-0.10000,8.51,293.0,-5.0,0.1800,-0.05,...,0.001000,0.001000,1.760000,1.690000,-0.020000,-0.020000,-0.002000,-0.002000,-0.010000,-0.010000
1,H-1 Above Plant,2009-01-14 10:15:57,9.36,1.4000,0.07000,8.36,294.0,2.0,0.2473,0.00,...,0.002152,0.002152,1.693529,1.693529,0.004336,0.012933,0.000644,0.000704,0.010771,0.010771
2,H-1 Above Plant,2009-02-04 15:28:29,10.34,1.5000,0.04000,8.25,375.0,6.0,0.2661,0.00,...,0.001326,0.001326,1.788837,1.688618,0.001143,0.002799,0.001598,0.000777,0.003766,0.003766
3,H-1 Above Plant,2009-03-04 15:33:29,10.28,1.2000,0.09300,8.41,309.0,6.0,0.1510,0.65,...,0.001472,0.001457,1.823606,1.823606,0.006627,0.001433,0.001058,0.001255,-0.003347,0.001787
4,H-1 Above Plant,2009-03-19 12:58:58,8.20,1.2000,0.03100,4.41,308.0,4.0,0.1370,0.00,...,0.001834,0.001834,1.767998,1.756106,0.002067,0.003839,0.001077,0.000436,0.001992,-0.005497
5,H-1 Above Plant,2009-04-07 09:31:57,9.74,2.4000,0.07100,8.34,339.0,3.0,0.1164,0.00,...,0.001916,0.001910,1.757605,1.757605,0.006020,-0.001359,0.001466,0.000881,-0.001148,0.000536
6,H-1 Above Plant,2009-04-30 14:03:20,9.58,3.4000,0.09000,8.59,266.0,17.0,0.1884,0.00,...,0.001503,0.001503,1.309299,1.285520,0.002774,0.000986,0.001244,0.000618,-0.000877,0.001013
7,H-1 Above Plant,2009-05-12 14:06:21,10.41,1.1000,0.05900,8.46,197.0,54.0,0.3502,0.00,...,0.001790,0.000784,1.238131,0.846946,0.000932,-0.001898,0.000747,0.000094,0.005131,0.001439
8,H-1 Above Plant,2009-06-05 07:54:32,9.10,1.1000,0.00700,8.49,194.0,NaN,0.3510,0.00,...,0.001195,0.001170,1.042242,0.794496,0.001989,0.003595,0.001265,0.000472,0.005421,0.005421
9,H-1 Above Plant,2009-07-22 09:14:54,8.53,0.8000,0.03000,8.48,216.0,14.0,0.1330,0.00,...,0.000895,0.000890,1.043110,0.968117,0.002314,-0.007751,0.000961,0.000434,0.002340,-0.000390


In [4]:
# Read the Huntington River Water Quality data - Dennis Oakley.xlsx data
column_list2 = []
PC2_cross_cols = pd.read_excel("Huntington River Water Quality data - Dennis Oakley.xlsx", sheet_name='Data ').columns
for i in PC2_cross_cols:
    column_list2.append(i) # list of column headers
converter2 = {col: float for col in column_list2[2:]} # convert most cols to float 
blanks = ('',' ','NA','#N/A','.','*','--','?','a','n','na','n/a','n.a.','n.d.',
          'nd','nm','nr','ns','x','Err','MD','N','NC','NA','ND','NDD','NE','NM',
          'NS','N/A','#VALUE!','not in','not  in','nm')
df_PC2_cross = pd.read_excel("Huntington River Water Quality data - Dennis Oakley.xlsx", sheet_name='Data ', skiprows=0,
                   header=0, converters=converter2, na_values=blanks)

old_colnames2 = ('Client Ident',  'Started',
 '0751\nMG/L\nOxygen, Dissolved\nOxygen, Dissolved\nMilligrams per Liter',
 '0751\nMG/L\nOil and Grease (HEM)\nOil and Grease, (HEM)\nMilligrams per Liter',
 '0752\nMG/L\nNitrogen, Ammonia\nNitrogen, Ammonia\nMilligrams per Liter',
 '9999\nS.U.\npH\npH\nStandard Units',
 '0844\nMG/L\nSolids, Total Dissolved\nTotal Dissolved Solids\nMilligrams per Liter',
 '0844\nMG/L\nSolids, Total Suspended\nTotal Suspended Solids\nMilligrams per Liter',
 '0751\nMG/L\nInorganic Anions\nNitrate\nMilligrams per Liter',
 '0751\nMG/L\nInorganic Anions\nNitrite\nMilligrams per Liter',
 'Nitrate + Nitrate as Nitrogen',
 'Ortho-Phosphate-P',
 '5531\nUG/L\nMercury, Hg\nMercury, Hg - Total\nMicrograms per Liter',
 '5531\nUG/L\nMercury, Hg\nMercury, Hg - Dissolved\nMicrograms per Liter',
 '5651\nMG/L\nPhosphorus, P\nPhosphorus, Dissolved\nMilligrams per Liter',
 '5651\nMG/L\nPhosphorus, P\nPhosphorus, Total\nMilligrams per Liter',
 '5021\nMG/L\nAqueous Metals\nAluminum, Al - Total\nMilligrams per Liter',
 '5021\nMG/L\nAqueous Metals\nAluminum, Al - Dissolved\nMilligrams per Liter',
 '5061\nMG/L\nAqueous Metals\nArsenic, As - Total\nMilligrams per Liter',
 '5061\nMG/L\nAqueous Metals\nArsenic, As - Dissolved\nMilligrams per Liter',
 '0512\nMG/L\nAqueous Metals\nBoron, B - Total\nMilligrams per Liter',
 '0012\nMG/L\nAqueous Metals\nBoron, B - Dissolved\nMilligrams per Liter',
 '5141\nMG/L\nAqueous Metals\nCadmium, Cd - Total\nMilligrams per Liter',
 '5141\nMG/L\nAqueous Metals\nCadmium, Cd - Dissolved\nMilligrams per Liter',
 '5151\nMG/L\nAqueous Metals\nCalcium, Ca - Total\nMilligrams per Liter',
 '5151\nMG/L\nAqueous Metals\nCalcium, Ca - Dissolved\nMilligrams per Liter',
 '5211\nMG/L\nAqueous Metals\nChromium, Cr - Total\nMilligrams per Liter',
 '5211\nMG/L\nAqueous Metals\nChromium, Cr - Dissolved\nMilligrams per Liter',
 '5231\nMG/L\nAqueous Metals\nCopper, Cu - Total\nMilligrams per Liter',
 '5231\nMG/L\nAqueous Metals\nCopper, Cu - Dissolved\nMilligrams per Liter',
 '5431\nMG/L\nAqueous Metals\nIron, Fe - Total\nMilligrams per Liter',
 '5431\nMG/L\nAqueous Metals\nIron, Fe - Dissolved\nMilligrams per Liter',
 '5471\nMG/L\nAqueous Metals\nLead, Pb - Total\nMilligrams per Liter',
 '5471\nMG/L\nAqueous Metals\nLead, Pb - Dissolved\nMilligrams per Liter',
 '5501\nMG/L\nAqueous Metals\nMagnesium, Mg - Total\nMilligrams per Liter',
 '5501\nMG/L\nAqueous Metals\nMagnesium, Mg - Dissolved\nMilligrams per Liter',
 '5581\nMG/L\nAqueous Metals\nNickel, Ni - Total\nMilligrams per Liter',
 '5581\nMG/L\nAqueous Metals\nNickel, Ni - Dissolved\nMilligrams per Liter',
 '5691\nMG/L\nAqueous Metals\nPotassium, K - Total\nMilligrams per Liter',
 '5691\nMG/L\nAqueous Metals\nPotassium, K - Dissolved\nMilligrams per Liter',
 '5811\nMG/L\nAqueous Metals\nSelenium, Se - Total\nMilligrams per Liter',
 '5811\nMG/L\nAqueous Metals\nSelenium, Se - Dissolved\nMilligrams per Liter',
 '5831\nMG/L\nAqueous Metals\nSilver, Ag - Total\nMilligrams per Liter',
 '5831\nMG/L\nAqueous Metals\nSilver, Ag - Dissolved\nMilligrams per Liter',
 '6021\nMG/L\nAqueous Metals\nZinc, Zn - Total\nMilligrams per Liter',
 '6021\nMG/L\nAqueous Metals\nZinc, Zn - Dissolved\nMilligrams per Liter')

new_colnames2 =('ClientID', 'Started', 'DO', 'O&G Hem', 'NH3-N', 'ph', 
 'TDS', 'TSS', 'NO3', 'NO2', 'NO2_3-N', 'OP-P', 'TotHg', 'DisHg', 'DisP', 'TotP', 'TotAl', 'DisAl', 'TotAs',
 'DisAs', 'TotB', 'DisB', 'TotCd', 'DisCd', 'TotCa', 'DisCa', 'TotCr', 'DisCr', 'TotCu', 'DisCu',
 'TotFe', 'DisFe', 'TotPb', 'DisPb', 'TotMg', 'DisMg', 'TotNi', 'DisNi', 'TotK', 'DisK', 'TotSe',
 'DisSe', 'TotAg', 'DisAg', 'TotZn', 'DisZn') # 'Temp', 'Source', 'LabId', 'SampleType', 'Client', 

df_PC2_cross.rename(columns={i:j for i,j in zip(old_colnames2,new_colnames2)}, inplace=True) # change the column headers
cols = df_PC2_cross.columns.drop('ClientID') # since string, keep col as object
df_PC2_cross[cols] = df_PC2_cross[cols].apply(pd.to_numeric, errors='coerce') # convert cols to floats, not objects
df_PC2_cross['Started'] = pd.to_datetime(df_PC2_cross['Started']) # Convert Date column to datetime

In [5]:
# Read the 2019 Huntington River Water Quality data - Dennis Oakley.xlsx data
column_list3 = []
PC3_cross_cols = pd.read_excel("Huntington River Water Quality data - Dennis Oakley.xlsx", sheet_name='2019').columns
for i in PC3_cross_cols:
    column_list3.append(i) # list of column headers
converter3 = {col: float for col in column_list3[6:]} # convert most cols to float 
blanks = ('',' ','NA','#N/A','.','*','--','?','a','n','na','n/a','n.a.','n.d.','nd','nm','nr','ns','x','Err','MD','N',
          'NC','NA','ND','NDD','NE','NM','NS','N/A','#VALUE!','not in','not  in','nm',
          '1F','A','A1','A5','A7','A9','AZ','BLK','CH','CR','D9','E4','G','G2','H','K','KN','LJ','M','r','RG','S','X','Z')
df_PC3_cross = pd.read_excel("Huntington River Water Quality data - Dennis Oakley.xlsx", sheet_name='2019', skiprows=0,
                   header=0, converters=converter3, na_values=blanks)
                
old_colnames3 = ('LabId',
 'SampleType',
 'Client',
 'ClientID',
 'Started',
 'Completed',
 '0110\nMG/L\nAQUA_SM2340B_HRD\nHARDNESS\nmg/L\n0',
 '0150\nUMHOS/CM\nAQUA_SM2510_COND\nCONDUCTIVITY\nµmhos/cm\n0.0',
 '0751\n---\nAQUA_1664_OIL_GREASE\nDISH_ID\n---\n0',
 '0751\nMG/L\nAQUA_1664_OIL_GREASE\nOIL_GREASE_HEM\nmg/L\n0',
 '0751\nMG/L\nAQUA_300.0_ANIONS\nNITRATE\nmg/L\n0.00',
 '0751\nMG/L\nAQUA_OXYGEN_DISS\nDISSOLVED_OXYGEN\nmg/L\n0.0',
 '0751\n%\nAQUA_ANION_CATION\nBALANCE\n%\n0.00',
 '0751\nMG/L\nAQUA_1664_OIL_GREASE\nOIL_GREASE_HEM\nmg/L\n02',
 '0751\nMG/L\nAQUA_D1067B_ACD\nACIDITY\nmg/L\n0.00',
 '0751\nMG/L\nAQUA_300.0_ANIONS\nNITRATE\nmg/L\n0.003',
 '0752\nMG/L\nAQUA_300.0_ANIONS\nNITRITE\nmg/L\n0.00',
 '0752\nMG/L\nAQUA_AMMONIA_SM\nAMMONIA\nmg/L\n0',
 '0752\nMG/L\nAQUA_SM2320_ALK\nAKC\nmg/L\n0.00',
 '0752\nMG/L\nAQUA_300.0_ANIONS\nNITRITE\nmg/L\n0.004',
 '0752\nMG/L\nAQUA_SM2320_ALK\nAK4\nmg/L\n0.00',
 '0754\nMG/L\nAQUA_300.0_ANIONS\nCHLORIDE\nmg/L\n0.0',
 '0755\nMG/L\nAQUA_300.0_ANIONS\nSULFATE\nmg/L\n0.0',
 '0844\n---\nAQUA_SM2540C_TDS\nDISH_ID\n---\n0',
 '0846\nG\nAQUA_SM2540C_TDS\nDISH_WEIGHT\nGrams\n0.0000',
 '0848\nG\nAQUA_SM2540C_TDS\nDISH+RESIDUE_WT\nGrams\n0.0000',
 '0850\nML\nAQUA_SM2540C_TDS\nSAMPLE_VOLUME\nmL\n0.00',
 '0852\nMG/L\nAQUA_SM2540C_TDS\nDISSOLVED_SOLIDS\nmg/L\n0.0',
 '0854\n---\nAQUA_SM2540D_TSS\nDISH_ID\n---\n0',
 '0856\nG\nAQUA_SM2540D_TSS\nFILTER_WEIGHT\nGrams\n0.0000',
 '0858\nG\nAQUA_SM2540D_TSS\nFILTER+RESIDUE_WT\nGrams\n0.0000',
 '0860\nML\nAQUA_SM2540D_TSS\nSAMPLE_VOLUME\nmL\n0.00',
 '0862\nMG/L\nAQUA_SM2540D_TSS\nSUSPENDED_SOLIDS\nmg/L\n0.0',
 '0999\n---\nAQUA_SM4500H_PH\nCONTAINER_ID\n---\n0',
 '5021\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_AL\nmg/L\n0.0',
 '5021\nMG/L\nAQUA_200.7_METALS\nTOTAL_AL\nmg/L\n0.0',
 '5061\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_AS\nmg/L\n0.000',
 '5061\nMG/L\nAQUA_200.7_METALS\nTOTAL_AS\nmg/L\n0.000',
 '5071\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_B\nmg/L\n0.00',
 '5071\nMG/L\nAQUA_200.7_METALS\nTOTAL_B\nmg/L\n0.00',
 '5141\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_CD\nmg/L\n0.000',
 '5141\nMG/L\nAQUA_200.7_METALS\nTOTAL_CD\nmg/L\n0.000',
 '5151\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_CA\nmg/L\n0',
 '5151\nMG/L\nAQUA_200.7_METALS\nTOTAL_CA\nmg/L\n0',
 '5211\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_CR\nmg/L\n0.00',
 '5211\nMG/L\nAQUA_200.7_METALS\nTOTAL_CR\nmg/L\n0.00',
 '5231\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_CU\nmg/L\n0.00',
 '5231\nMG/L\nAQUA_200.7_METALS\nTOTAL_CU\nmg/L\n0.00',
 '5431\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_FE\nmg/L\n0.00',
 '5431\nMG/L\nAQUA_200.7_METALS\nTOTAL_FE\nmg/L\n0.00',
 '5471\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_PB\nmg/L\n0.00',
 '5471\nMG/L\nAQUA_200.7_METALS\nTOTAL_PB\nmg/L\n0.00',
 '5501\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_MG\nmg/L\n0',
 '5501\nMG/L\nAQUA_200.7_METALS\nTOTAL_MG\nmg/L\n0',
 '5511\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_MN\nmg/L\n0.00',
 '5511\nMG/L\nAQUA_200.7_METALS\nTOTAL_MN\nmg/L\n0.00',
 '5531\nUG/L\nAQUA_245.1_HG\nDISSOLVED\nµg/L\n0.000',
 '5531\nUG/L\nAQUA_245.1_HG\nTOTAL\nµg/L\n0.0000',
 '5581\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_NI\nmg/L\n0.00',
 '5581\nMG/L\nAQUA_200.7_METALS\nTOTAL_NI\nmg/L\n0.00',
 '5651\nMG/L\nAQUA_SM4500PE\nDISSOLVED_P\nmg/L\n0.00',
 '5651\nMG/L\nAQUA_SM4500PE\nTOTAL_P\nmg/L\n0.00',
 '5691\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_K\nmg/L\n0',
 '5691\nMG/L\nAQUA_200.7_METALS\nTOTAL_K\nmg/L\n0',
 '5811\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_SE\nmg/L\n0.00',
 '5811\nMG/L\nAQUA_200.7_METALS\nTOTAL_SE\nmg/L\n0.00',
 '5831\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_AG\nmg/L\n0.00',
 '5831\nMG/L\nAQUA_200.7_METALS\nTOTAL_AG\nmg/L\n0.00',
 '5841\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_NA\nmg/L\n0',
 '6021\nMG/L\nAQUA_200.7_METALS\nDISSOLVED_ZN\nmg/L\n0.00',
 '6021\nMG/L\nAQUA_200.7_METALS\nTOTAL_ZN\nmg/L\n0.00',
 '9999\nS.U.\nAQUA_SM4500H_PH\nPH\ns. u.\n0.00',
 '9999\n°C\nAQUA_SM4500H_PH\nPH_TEMPERATURE\n°C\n0.00')

new_colnames3 = ('LabId','SampleType','Client','ClientID','Started','Completed',
                 'Hard','Cond','O&G Dish','O&G Hem','NO3','DO','An-Cat Bal','O&G Hem2','Acid','NO32','NO2','NH3','Alk',
                 'NO22','Alk2','TotCl','TotSO4','TDSDishID','TDSdishWt','TDSdish_resWt','TDSVol','TDS','TSSDishID',
                 'TSSfiltWt', 'TSSfilt_resWt','TSSVol','TSS','pHID','DisAl','TotAl','DisAs','TotAs','DisB','TotB',
                 'DisCd','TotCd', 'DisCa','TotCa','DisCr','TotCr','DisCu', 'TotCu', 'DisFe', 'TotFe','DisPb','TotPb',
                 'DisMg','TotMg','DisMn','TotMn','DisHg','TotHg','DisNi','TotNi','DisP','TotP','DisK','TotK','DisSe',
                 'TotSe','DisAg','TotAg','DisNa','DisZn','TotZn','pH','Temp')
# Combine O&G Hem2 into O&G Hem, NO32 into NO3, NO22 into NO2,  Alk2 into Alk, 

df_PC3_cross.rename(columns={i:j for i,j in zip(old_colnames3,new_colnames3)}, inplace=True) # change the column headers
cols = df_PC3_cross.columns.drop(['LabId','SampleType','Client','ClientID','Started','Completed']) # since string, keep col as object
df_PC3_cross[cols] = df_PC3_cross[cols].apply(pd.to_numeric, errors='coerce') # convert cols to floats, not objects
df_PC3_cross['Started'] = pd.to_datetime(df_PC3_cross['Started']) # Convert Date column to datetime
df_PC3_cross['Completed'] = pd.to_datetime(df_PC3_cross['Completed']) # Convert Date column to datetime
    
df_PC2_cross
df_PC3_cross

,LabId,SampleType,Client,ClientID,Started,Completed,Hard,Cond,O&G Dish,O&G Hem,...,TotK,DisSe,TotSe,DisAg,TotAg,DisNa,DisZn,TotZn,pH,Temp
0,1913258-001,UNK,141687 PACIFICORP,UPL-3 Below Dam,2019-01-31,2019-02-27,NaN,NaN,2.0,1.0,...,1.0,0.00,0.00,0.00,0.03,NaN,0.00,0.00,7.94,10.9
1,1913203-001,UNK,141687 PACIFICORP,UPL-1 Left Fork,2019-01-31,2019-03-08,NaN,NaN,4.0,0.0,...,1.0,0.00,0.00,0.00,0.00,NaN,0.00,0.00,8.64,11.9
2,1913203-002,UNK,141687 PACIFICORP,UPL-2 Right Fork,2019-01-31,2019-03-08,NaN,NaN,5.0,0.0,...,1.0,0.00,0.00,0.00,0.00,NaN,0.00,0.00,8.59,11.4
3,1913655-001,UNK,141687 PACIFICORP,UPL-1 Left Fork,2019-02-14,2019-03-08,NaN,NaN,11.0,0.0,...,1.0,0.00,0.00,-0.01,0.00,NaN,0.00,-0.08,8.32,19.0
4,1913655-002,UNK,141687 PACIFICORP,UPL-2 Right Fork,2019-02-14,2019-03-08,NaN,NaN,12.0,0.0,...,2.0,0.00,0.00,-0.01,0.00,NaN,0.00,-0.08,8.22,18.9
5,1913655-003,UNK,141687 PACIFICORP,UPL-3 Below Dam,2019-02-14,2019-03-08,NaN,NaN,13.0,0.0,...,1.0,0.00,0.00,0.00,0.00,NaN,0.00,-0.07,7.96,18.9
6,1914485-003,UNK,141687 PACIFICORP,UPL-1 Left Fork,2019-03-21,2019-04-08,NaN,NaN,4.0,NaN,...,1.0,0.00,0.00,0.00,0.00,NaN,-0.02,0.00,8.49,13.1
7,1914485-004,UNK,141687 PACIFICORP,UPL-2 Right Fork,2019-03-21,2019-04-08,NaN,NaN,5.0,NaN,...,2.0,0.04,0.00,-0.02,0.00,NaN,0.00,0.00,8.37,13.0
8,1914485-005,UNK,141687 PACIFICORP,UPL-3 Below Dam,2019-03-21,2019-04-08,NaN,NaN,6.0,NaN,...,1.0,0.04,0.02,0.00,0.00,NaN,0.00,0.00,8.00,13.8
9,1915287-001,UNK,141687 PACIFICORP,UPL-1 Left Fork,2019-04-18,2019-05-01,NaN,NaN,2.0,0.0,...,1.0,-0.02,-0.02,-0.01,0.17,NaN,0.00,-0.02,8.52,11.0


### Append the Crosstable datasets into single dataframe

In [6]:
df_cross = df_PC3_cross.append(df_PC2_cross, ignore_index=True, sort=False)
df_cross = df_cross.append(df_PC_cross, ignore_index=True, sort=False)
df_cross[df_cross.columns[6:-1]] = df_cross[df_cross.columns[6:-1]].abs() # Remove neg values as indicates "less than DL"
# pd.set_option('display.max_rows', None)
df = df_cross.copy() # copy the dataframe for clarity
df = df.sort_values(by='ClientID',ascending=True) # sort the table by Client ID ascending
df

,LabId,SampleType,Client,ClientID,Started,Completed,Hard,Cond,O&G Dish,O&G Hem,...,TotAg,DisNa,DisZn,TotZn,pH,Temp,NH3-N,ph,NO2_3-N,OP-P
837,NaN,NaN,NaN,H-1,2015-03-26 00:00:00,NaT,NaN,NaN,NaN,5.0000,...,0.002000,NaN,0.004000,0.004000,NaN,NaN,0.10000,8.35,NaN,NaN
987,NaN,NaN,NaN,H-1,2016-06-21 00:00:00,NaT,NaN,NaN,NaN,5.0000,...,0.002000,NaN,0.004000,0.004000,NaN,NaN,0.20000,8.45,NaN,NaN
926,NaN,NaN,NaN,H-1,2015-11-09 00:00:00,NaT,NaN,NaN,NaN,5.0000,...,0.002000,NaN,0.004000,0.004000,NaN,NaN,0.10000,8.42,NaN,NaN
706,NaN,NaN,NaN,H-1,2014-03-25 00:00:00,NaT,NaN,NaN,NaN,5.0000,...,0.002000,NaN,0.004000,0.004000,NaN,NaN,0.10000,8.50,NaN,NaN
1143,NaN,NaN,NaN,H-1,2018-03-19 00:00:00,NaT,NaN,NaN,NaN,5.0000,...,0.014000,NaN,0.004000,0.004000,NaN,NaN,0.10000,8.39,NaN,NaN
1168,NaN,NaN,NaN,H-1 (HTG Creek @ River Intake),2009-12-06 00:00:00,NaT,NaN,NaN,NaN,5.0000,...,0.002000,NaN,0.010000,0.010000,NaN,NaN,0.10000,8.51,NaN,NaN
225,NaN,NaN,NaN,H-1 (HTG Creek @ River Intake),2009-12-06 00:00:00,NaT,NaN,NaN,NaN,5.0000,...,0.002000,NaN,0.010000,0.010000,NaN,NaN,0.10000,8.51,NaN,NaN
1199,NaN,NaN,NaN,H-1 Above Plant,2011-05-02 00:00:00,NaT,NaN,NaN,NaN,5.0000,...,0.002000,NaN,0.009000,0.009000,NaN,NaN,0.10000,8.48,NaN,NaN
157,NaN,NaN,NaN,H-1 Above Plant,2009-09-11 17:03:45,NaT,NaN,NaN,NaN,1.2000,...,0.001301,NaN,0.001799,0.001877,NaN,NaN,0.03965,8.52,NaN,NaN
1201,NaN,NaN,NaN,H-1 Above Plant,2011-06-13 00:00:00,NaT,NaN,NaN,NaN,5.0000,...,0.002000,NaN,0.004000,0.004000,NaN,NaN,0.10000,8.45,NaN,NaN


### Add a lookup dictionary for similar SiteID

In [7]:
# Read in lookup table with ClientID, combined SiteID, Lat, Long, Downstream Order (No), and mileage down (DOWN_mi)
df_lookup = pd.read_excel("SiteLU.xlsx", sheet_name="Sheet1", skiprows=0, header=0) # Read table in
df3 = pd.merge(left=df,right=df_lookup,left_on=['ClientID'], right_on = ['ClientID'], how = 'left') # merge lookup to df
df3 = df3.sort_values(by='ClientID',ascending=True) # sort the table by Client ID ascending
df3

,LabId,SampleType,Client,ClientID,Started,Completed,Hard,Cond,O&G Dish,O&G Hem,...,Temp,NH3-N,ph,NO2_3-N,OP-P,SiteID,LAT,LONG,No,DOWN_mi
0,NaN,NaN,NaN,H-1,2015-03-26 00:00:00,NaT,NaN,NaN,NaN,5.0,...,NaN,0.10000,8.35,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
1,NaN,NaN,NaN,H-1,2015-03-26 00:00:00,NaT,NaN,NaN,NaN,5.0,...,NaN,0.10000,8.35,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
2,NaN,NaN,NaN,H-1,2016-06-21 00:00:00,NaT,NaN,NaN,NaN,5.0,...,NaN,0.20000,8.45,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
3,NaN,NaN,NaN,H-1,2016-06-21 00:00:00,NaT,NaN,NaN,NaN,5.0,...,NaN,0.20000,8.45,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
4,NaN,NaN,NaN,H-1,2015-11-09 00:00:00,NaT,NaN,NaN,NaN,5.0,...,NaN,0.10000,8.42,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
5,NaN,NaN,NaN,H-1,2015-11-09 00:00:00,NaT,NaN,NaN,NaN,5.0,...,NaN,0.10000,8.42,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
6,NaN,NaN,NaN,H-1,2014-03-25 00:00:00,NaT,NaN,NaN,NaN,5.0,...,NaN,0.10000,8.50,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
7,NaN,NaN,NaN,H-1,2014-03-25 00:00:00,NaT,NaN,NaN,NaN,5.0,...,NaN,0.10000,8.50,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
8,NaN,NaN,NaN,H-1,2018-03-19 00:00:00,NaT,NaN,NaN,NaN,5.0,...,NaN,0.10000,8.39,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
9,NaN,NaN,NaN,H-1,2018-03-19 00:00:00,NaT,NaN,NaN,NaN,5.0,...,NaN,0.10000,8.39,NaN,NaN,H-1,39.391714,-111.093796,12,26.77


### Clean up the df

In [8]:
df3.rename(columns={'LAT':'Lat', 'LONG':'Long', 'DOWN_mi':'Mileage'}, inplace=True) # change header names
df3.drop(['LabId','SampleType','Client','Completed'], axis = 1, inplace = True) # drop unnecessary columns
df4 = df3.copy()
df4

,ClientID,Started,Hard,Cond,O&G Dish,O&G Hem,NO3,DO,An-Cat Bal,O&G Hem2,...,Temp,NH3-N,ph,NO2_3-N,OP-P,SiteID,Lat,Long,No,Mileage
0,H-1,2015-03-26 00:00:00,NaN,NaN,NaN,5.0,0.2100,10.70,NaN,NaN,...,NaN,0.10000,8.35,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
1,H-1,2015-03-26 00:00:00,NaN,NaN,NaN,5.0,0.2100,10.70,NaN,NaN,...,NaN,0.10000,8.35,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
2,H-1,2016-06-21 00:00:00,NaN,NaN,NaN,5.0,0.1200,7.90,NaN,NaN,...,NaN,0.20000,8.45,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
3,H-1,2016-06-21 00:00:00,NaN,NaN,NaN,5.0,0.1200,7.90,NaN,NaN,...,NaN,0.20000,8.45,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
4,H-1,2015-11-09 00:00:00,NaN,NaN,NaN,5.0,0.1900,9.70,NaN,NaN,...,NaN,0.10000,8.42,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
5,H-1,2015-11-09 00:00:00,NaN,NaN,NaN,5.0,0.1900,9.70,NaN,NaN,...,NaN,0.10000,8.42,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
6,H-1,2014-03-25 00:00:00,NaN,NaN,NaN,5.0,0.2500,13.30,NaN,NaN,...,NaN,0.10000,8.50,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
7,H-1,2014-03-25 00:00:00,NaN,NaN,NaN,5.0,0.2500,13.30,NaN,NaN,...,NaN,0.10000,8.50,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
8,H-1,2018-03-19 00:00:00,NaN,NaN,NaN,5.0,0.2400,8.50,NaN,NaN,...,NaN,0.10000,8.39,NaN,NaN,H-1,39.391714,-111.093796,12,26.77
9,H-1,2018-03-19 00:00:00,NaN,NaN,NaN,5.0,0.2400,8.50,NaN,NaN,...,NaN,0.10000,8.39,NaN,NaN,H-1,39.391714,-111.093796,12,26.77


### Get stats of the analytes for all sites

In [12]:
df4[["TDS", "TotFe", "DisFe"]].describe() # basic stats over all of the sites for each analyte

,TDS,TotFe,DisFe
count,3894.000000,3892.000000,3892.000000
mean,223.281715,0.465368,0.031092
std,77.500651,2.729137,0.066788
min,26.000000,0.000000,0.000000
25%,169.000000,0.050000,0.030000
50%,201.000000,0.136594,0.030000
75%,248.000000,0.310000,0.030000
max,763.000000,48.220000,1.690000


In [14]:
# function for getting percentiles with other agg describe statistics
def per(n):
    def percentile_(x):
        return x.quantile(n)
    percentile_.__name__ = '{:02.0f}%'.format(n*100)
    return percentile_

# stats for grouped SiteIDs only for TDS
analyte = ["TDS","TotFe","DisFe"]
print ("Describe statistics for TDS mg/l at each site")
sites = (df4.groupby("SiteID")["TDS"].agg(["count","min",per(.20),"mean","median",per(.80),"max","std","var","skew"])).round(2)
sites

# # Make a pretty table
# table = ff.create_table(sites)
# py.iplot(table, filename='jupyter-table1')

Describe statistics for TDS mg/l at each site


,count,min,20%,mean,median,80%,max,std,var,skew
SiteID,,,,,,,,,,
H-1,362,26.0,220.0,277.04,276.0,329.0,763.0,66.41,4410.21,1.94
H-2,528,168.0,227.0,300.24,284.0,362.0,515.0,79.22,6275.49,0.73
JC-1,30,196.0,210.0,217.10,219.0,224.2,228.0,9.14,83.47,-1.14
PG-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UPL-1,988,131.0,177.0,202.60,197.0,234.0,275.0,30.99,960.58,0.28
UPL-10,131,105.0,161.0,203.65,218.0,234.0,262.0,39.36,1549.08,-0.93
UPL-11,132,119.0,151.0,168.40,167.0,181.0,250.0,24.01,576.56,0.64
UPL-2,142,162.0,209.2,231.90,232.0,258.6,301.0,28.70,823.95,-0.05
UPL-3,142,137.0,175.0,188.34,188.0,201.0,244.0,19.06,363.15,0.08


In [15]:
# stats for grouped SiteIDs only for TotFe
analyte = ["TDS","TotFe","DisFe"]
print ("Describe statistics for TotFe mg/l at each site")
sites = (df4.groupby("SiteID")["TotFe"].agg(["count","min",per(.20),"mean","median",per(.80),"max","std","var","skew"])).round(2)
sites

Describe statistics for TotFe mg/l at each site


,count,min,20%,mean,median,80%,max,std,var,skew
SiteID,,,,,,,,,,
H-1,360,0.03,0.10,0.72,0.19,0.72,47.75,3.59,12.86,12.66
H-2,528,0.03,0.11,1.04,0.20,0.67,48.22,5.13,26.28,8.92
JC-1,30,0.21,0.21,0.23,0.24,0.24,0.26,0.02,0.00,0.21
PG-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UPL-1,988,0.00,0.07,0.35,0.17,0.43,7.82,0.81,0.65,6.93
UPL-10,131,0.08,0.24,0.39,0.36,0.45,3.26,0.29,0.09,7.53
UPL-11,132,0.03,0.16,0.28,0.21,0.37,1.12,0.18,0.03,2.20
UPL-2,142,0.00,0.15,0.67,0.28,0.72,12.01,1.49,2.22,6.49
UPL-3,142,0.00,0.06,0.22,0.12,0.22,3.11,0.50,0.25,5.48


In [16]:
# stats for grouped SiteIDs only for DisFe
analyte = ["TDS","TotFe","DisFe"]
print ("Describe statistics for DisFe mg/l at each site")
sites = (df4.groupby("SiteID")["DisFe"].agg(["count","min",per(.20),"mean","median",per(.80),"max","std","var","skew"])).round(2)
sites

Describe statistics for DisFe mg/l at each site


,count,min,20%,mean,median,80%,max,std,var,skew
SiteID,,,,,,,,,,
H-1,360,0.00,0.03,0.04,0.03,0.03,1.69,0.12,0.02,12.99
H-2,528,0.00,0.03,0.04,0.03,0.03,1.45,0.11,0.01,12.83
JC-1,30,0.03,0.03,0.03,0.03,0.03,0.04,0.00,0.00,2.81
PG-1,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UPL-1,988,0.00,0.03,0.03,0.03,0.03,0.37,0.03,0.00,9.00
UPL-10,131,0.00,0.03,0.04,0.03,0.06,0.43,0.05,0.00,5.43
UPL-11,132,0.00,0.03,0.04,0.03,0.04,0.26,0.03,0.00,4.35
UPL-2,142,0.00,0.03,0.03,0.03,0.03,0.30,0.04,0.00,5.53
UPL-3,142,0.00,0.03,0.03,0.03,0.03,0.34,0.03,0.00,6.73


### Convert crosstab dataframe to a flat file with melt

In [17]:
keep = ['ClientID','SiteID','Started','No','Mileage','Lat','Long'] # columns to keep with analyte and result
dfflat = df4.melt(id_vars=keep, var_name='Analyte', value_name='Result').dropna() # melt the crosstab
dfflat

,ClientID,SiteID,Started,No,Mileage,Lat,Long,Analyte,Result
9112,UPL-1 Left Fork,UPL-1,2019-04-18,2,0.11,39.690328,-111.259390,O&G Dish,2.00
9113,UPL-1 Left Fork,UPL-1,2019-04-18,2,0.11,39.690328,-111.259390,O&G Dish,2.00
9114,UPL-1 Left Fork,UPL-1,2019-04-18,2,0.11,39.690328,-111.259390,O&G Dish,2.00
9115,UPL-1 Left Fork,UPL-1,2019-04-18,2,0.11,39.690328,-111.259390,O&G Dish,2.00
9124,UPL-1 Left Fork,UPL-1,2019-04-18,2,0.11,39.690328,-111.259390,O&G Dish,2.00
9140,UPL-1 Left Fork,UPL-1,2019-04-18,2,0.11,39.690328,-111.259390,O&G Dish,2.00
9141,UPL-1 Left Fork,UPL-1,2019-03-21,2,0.11,39.690328,-111.259390,O&G Dish,4.00
9148,UPL-1 Left Fork,UPL-1,2019-02-14,2,0.11,39.690328,-111.259390,O&G Dish,11.00
9149,UPL-1 Left Fork,UPL-1,2019-02-14,2,0.11,39.690328,-111.259390,O&G Dish,11.00
9150,UPL-1 Left Fork,UPL-1,2019-02-14,2,0.11,39.690328,-111.259390,O&G Dish,11.00


### Make a map of all of the SiteIDs

In [68]:
from numpy import mean
fig = go.Figure()

dfn = dfflat[dfflat['Analyte'] == "TDS"].sort_values('Started', ascending=False) # TDS subset sort by new to old date
dfe = dfn.drop_duplicates('SiteID') # most recent chosen analyte for all wells in chosen project
dfe[["Result"]] = dfe[["Result"]].apply(pd.to_numeric) # change Results to numeric value
num = [str(i) for i in dfe['No']]
mil = [str(j) for j in dfe['Mileage']]
res = [str(k) for k in dfe['Result']]

fig.add_trace(go.Scattermapbox(
    name='Project Wells',
    lat=dfe['Lat'], lon=dfe['Long'], mode='markers',
    marker=go.scattermapbox.Marker(
        size=dfe['Result']*0.1, # size=17,
        color=dfe['Result'], #np.log(dfe['Result']), #'Red', # 'LightSkyBlue',
        showscale=True,
        colorbar=dict(
            title="TDS mg/L",
            thicknessmode="pixels", thickness=20, lenmode="pixels",
            len=400, yanchor="bottom", y=0, ticks="outside", 
            ticksuffix=" mg/l"),
        colorscale='Viridis',
        cmin=dfe['Result'].min(),
        cmax=dfe['Result'].max()),
    text=dfe.SiteID + "<br>" + "No:" + num + "<br>"+ "Mile:" + mil + "<br>" + res + " mg/l",
    hoverinfo='text')), 

fig.update_layout(mapbox_style="white-bg",
                  autosize=True, # not zooming in
                  showlegend=True,
                  mapbox=dict(
#                       accesstoken=mapbox_access_token,
                      bearing=0,
                      center=dict(lat=mean(dfflat.Lat), lon=mean(dfflat.Long)),
                      pitch=0,zoom=10),
                  mapbox_layers=[{
                      "below": 'traces',
                      "sourcetype": "raster",
                      "source": ["https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"]}
                  ])

fig.update_layout(#height=300, margin={"r":2, "t":2, "l":2, "b":2}, #margin= {'t': 60, 'b': 20},
    title_text='Map of each SiteID TDS color and size',
    height=600, margin= {'t': 60, 'b': 20},
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)') # margin={"r":0,"t":0,"l":0,"b":0}

fig.show()

In [72]:
from numpy import mean
fig = go.Figure()

dfn = dfflat[dfflat['Analyte'] == "TotFe"].sort_values('Started', ascending=False) # TotFe subset sort by new to old date
dfe = dfn.drop_duplicates('SiteID') # most recent chosen analyte for all wells in chosen project
dfe[["Result"]] = dfe[["Result"]].apply(pd.to_numeric) # change Results to numeric value
num = [str(i) for i in dfe['No']]
mil = [str(j) for j in dfe['Mileage']]
res = [str(k) for k in dfe['Result']]

fig.add_trace(go.Scattermapbox(
    name='Project Wells',
    lat=dfe['Lat'], lon=dfe['Long'], mode='markers',
    marker=go.scattermapbox.Marker(
        size=dfe['Result']*100, # size=17,
        color=dfe['Result'], #np.log(dfe['Result']), #'Red', # 'LightSkyBlue',
        showscale=True,
        colorbar=dict(
            title="TotFe mg/L",
            thicknessmode="pixels", thickness=20, lenmode="pixels",
            len=400, yanchor="bottom", y=0, ticks="outside", 
            ticksuffix=" mg/l"),
        colorscale='Viridis',
        cmin=dfe['Result'].min(),
        cmax=dfe['Result'].max()),
    text=dfe.SiteID + "<br>" + "No:" + num + "<br>"+ "Mile:" + mil + "<br>" + res + " mg/l",
    hoverinfo='text')), 

fig.update_layout(mapbox_style="white-bg",
                  autosize=True, # not zooming in
                  showlegend=True,
                  mapbox=dict(
#                       accesstoken=mapbox_access_token,
                      bearing=0,
                      center=dict(lat=mean(dfflat.Lat), lon=mean(dfflat.Long)),
                      pitch=0,zoom=10),
                  mapbox_layers=[{
                      "below": 'traces',
                      "sourcetype": "raster",
                      "source": ["https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"]}
                  ])

fig.update_layout(#height=300, margin={"r":2, "t":2, "l":2, "b":2}, #margin= {'t': 60, 'b': 20},
    title_text='Map of each SiteID TotFe color and size',
    height=600, margin= {'t': 60, 'b': 20},
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)') # margin={"r":0,"t":0,"l":0,"b":0}

fig.show()

In [74]:
from numpy import mean
fig = go.Figure()

dfn = dfflat[dfflat['Analyte'] == "DisFe"].sort_values('Started', ascending=False) # DisFe subset sort by new to old date
dfe = dfn.drop_duplicates('SiteID') # most recent chosen analyte for all wells in chosen project
dfe[["Result"]] = dfe[["Result"]].apply(pd.to_numeric) # change Results to numeric value
num = [str(i) for i in dfe['No']]
mil = [str(j) for j in dfe['Mileage']]
res = [str(k) for k in dfe['Result']]

fig.add_trace(go.Scattermapbox(
    name='Project Wells',
    lat=dfe['Lat'], lon=dfe['Long'], mode='markers',
    marker=go.scattermapbox.Marker(
        size=dfe['Result']*1000, # size=17,
        color=dfe['Result'], #np.log(dfe['Result']), #'Red', # 'LightSkyBlue',
        showscale=True,
        colorbar=dict(
            title="DisFe mg/L",
            thicknessmode="pixels", thickness=20, lenmode="pixels",
            len=400, yanchor="bottom", y=0, ticks="outside", 
            ticksuffix=" mg/l"),
        colorscale='Viridis',
        cmin=dfe['Result'].min(),
        cmax=dfe['Result'].max()),
    text=dfe.SiteID + "<br>" + "No:" + num + "<br>"+ "Mile:" + mil + "<br>" + res + " mg/l",
    hoverinfo='text')), 

fig.update_layout(mapbox_style="white-bg",
                  autosize=True, # not zooming in
                  showlegend=True,
                  mapbox=dict(
#                       accesstoken=mapbox_access_token,
                      bearing=0,
                      center=dict(lat=mean(dfflat.Lat), lon=mean(dfflat.Long)),
                      pitch=0,zoom=10),
                  mapbox_layers=[{
                      "below": 'traces',
                      "sourcetype": "raster",
                      "source": ["https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryOnly/MapServer/tile/{z}/{y}/{x}"]}
                  ])

fig.update_layout(#height=300, margin={"r":2, "t":2, "l":2, "b":2}, #margin= {'t': 60, 'b': 20},
    title_text='Map of each SiteID DisFe color and size',
    height=600, margin= {'t': 60, 'b': 20},
    paper_bgcolor='rgba(0,0,0,0)', plot_bgcolor='rgba(0,0,0,0)') # margin={"r":0,"t":0,"l":0,"b":0}

fig.show()

### Make some barplots of TDS downstream

In [67]:
dfn = dfflat[dfflat['Analyte'] == "TDS"].sort_values('No') # Subset df only TDS and sort by site number (No)
dfo = dfflat[dfflat['Analyte'] == "TotFe"].sort_values('No') # Subset df only TotFe and sort by site number (No).
dfp = dfflat[dfflat['Analyte'] == "DisFe"].sort_values('No') # Subset df only DisFe and sort by site number (No)

fig = make_subplots(rows=3, cols=1, 
                    shared_xaxes=True, 
                    vertical_spacing=0.02)

fig.add_trace(go.Bar(x=dfn['SiteID'], y=dfn['Result'], # Simple bar plot of TDS values for each site
#                      marker=dict(color="red"),
                     marker_color="red", marker_line_color="red",
                     marker_line_width=1.5, opacity=0.3,
                     name='TDS'), row=1, col=1) # marker_color='rgb(55, 83, 109)')

fig.add_trace(go.Bar(x=dfo['SiteID'], y=dfo['Result'], # Simple bar plot of TotFe values for each site
                     marker_color="green", marker_line_color="green",
                     marker_line_width=1.5, opacity=0.3,
                     name='TotFe'), row=2, col=1)

fig.add_trace(go.Bar(x=dfp['SiteID'], y=dfp['Result'], # Simple bar plot of DisFe values for each site
                     marker_color="blue", marker_line_color="blue",
                     marker_line_width=1.5, opacity=0.3,
                     name='DisFe'), row=3, col=1)

fig.update_yaxes(title_text="TDS mg/L", row=1, col=1)
fig.update_yaxes(title_text="TotFe", type="log", row=2, col=1)
fig.update_yaxes(title_text="DisFe", type="log", row=3, col=1)
fig.update_layout(height=600, width=1000,
                  title_text="TDS, TotFe, and DisFe barplots with all data points")
fig.show()

### Create a violin and bar plot for each SiteID

In [60]:
dfn = dfflat[dfflat['Analyte'] == "TDS"].sort_values('No') # Subset df only TDS and sort by site number (No)
dfo = dfflat[dfflat['Analyte'] == "TotFe"].sort_values('No') # Subset df only TotFe and sort by site number (No).
dfp = dfflat[dfflat['Analyte'] == "DisFe"].sort_values('No') # Subset df only DisFe and sort by site number (No)
dsite = dfn['SiteID'].unique() # get unique SiteIDs list

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02) # plot with several subplots
for h in dsite:
    fig.add_trace(go.Violin(x=dfn['SiteID'][dfn['SiteID'] == h],
                            y=dfn['Result'][dfn['SiteID'] == h],
                            name=h, points='all', box_visible=True, meanline_visible=True), row=1, col=1)
    
    fig.add_trace(go.Violin(x=dfo['SiteID'][dfo['SiteID'] == h],
                            y=dfo['Result'][dfo['SiteID'] == h],
                            points='all', box_visible=True, meanline_visible=True), row=2, col=1)
    
    fig.add_trace(go.Violin(x=dfp['SiteID'][dfp['SiteID'] == h],
                            y=dfp['Result'][dfp['SiteID'] == h],
                            points='all', box_visible=True, meanline_visible=True), row=3, col=1)

    fig.update_yaxes(title_text="TDS mg/L", row=1, col=1)
    fig.update_yaxes(title_text="TotFe", type="log", row=2, col=1)
    fig.update_yaxes(title_text="DisFe", type="log", row=3, col=1)
    fig.update_layout(title_text='TDS, TotFe, and DisFe violin with all data points',
                      xaxis_showgrid=True, xaxis_zeroline=True,
                      height=800, margin= {'t': 60, 'b': 20},
                      plot_bgcolor='white')
    
fig.show()

### Make a trendline plot of each of the SiteIDs moving downstream

In [80]:
dfn = dfflat[dfflat['Analyte'] == "TDS"].sort_values('No').sort_values('Started') # pick just TDS sorted by number and date
dfo = dfflat[dfflat['Analyte'] == "TotFe"].sort_values('No').sort_values('Started') # pick just TotFe sorted by number and date
dfp = dfflat[dfflat['Analyte'] == "DisFe"].sort_values('No').sort_values('Started') # pick just DisFe sorted by number and date
dsite = dfflat['SiteID'].unique() # get unique SiteIDs for list

fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.02) # plot with several subplots
for i in dsite: # loop through each SiteID
    fig.add_trace(go.Scatter(x=dfn['Started'][dfn['SiteID'] == i],
                             y=dfn['Result'][dfn['SiteID'] == i],
                             name=i, hoverinfo='all'), row=1, col=1) # add TDS trendline for each SiteID
    
    fig.add_trace(go.Scatter(x=dfo['Started'][dfo['SiteID'] == i],
                             y=dfo['Result'][dfo['SiteID'] == i],
                             name=i, hoverinfo='all'), row=2, col=1) # add TotFe trendline for each SiteID
    
    fig.add_trace(go.Scatter(x=dfp['Started'][dfp['SiteID'] == i],
                             y=dfp['Result'][dfp['SiteID'] == i],
                             name=i, hoverinfo='all'), row=3, col=1) # add DisFe trendline for each SiteID

fig.update_yaxes(title_text="TDS mg/L", row=1, col=1)
fig.update_yaxes(title_text="TotFe", type="log", row=2, col=1)
fig.update_yaxes(title_text="DisFe", type="log", row=3, col=1)
fig.update_layout(title_text="TDS, TotFe, and DisFe trend plots with all data points", # add a figure title
                  xaxis_showgrid=True, xaxis_zeroline=True,
                  height=800, margin= {'t': 60, 'b': 20})

fig.show()

In [ ]:
# AQWMS_flat = pd.read_excel("Electric_Lake_Data_2011_2013_2019.xlsx")